## Load dataset

In [1]:
import numpy as np

from keras import backend as K
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras import optimizers

seed = 42

np.random.seed(seed)

Using TensorFlow backend.


In [2]:
from keras.datasets import cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

n_training_examples = x_train.shape[0]
n_testing_examples = x_test.shape[0]
print('Training data shape: {}'.format(x_train.shape))
print('Number of training examples: {}'.format(n_training_examples))
print('Number of testing examples: {}'.format(n_testing_examples))

Training data shape: (50000, 32, 32, 3)
Number of training examples: 50000
Number of testing examples: 10000


## Set hyperparameters

In [3]:
batch_size = 16
n_epochs = 30
n_classes = 100

## Data generator

In [4]:
from keras.applications.resnet50 import preprocess_input
import cv2

def image_generator(images, labels):
    def generator():
        start = 0
        end = batch_size
        n = len(images)
        while True:
            img_batch_resized = np.array([cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC) for img in images[start:end]])
            img_batch_resized = img_batch_resized.astype('float32')
            img_batch_preprocessed = preprocess_input(img_batch_resized)
            label_batch = labels[start:end]
            
            start += batch_size
            end += batch_size
            if start >= n:
                start = 0
                end = batch_size
            yield (img_batch_preprocessed, label_batch)
    return generator

## Pre-trained model

In [5]:
from keras.applications.resnet50 import ResNet50
pretrained_model = ResNet50(weights='imagenet', include_top=False)

## Extract features

In [7]:
# K-fold cross validation
from sklearn.model_selection import StratifiedKFold

y_train = y_train.flatten()
y_test = y_test.flatten()

X = np.concatenate([x_train, x_test], axis=0)
Y = np.concatenate([y_train, y_test], axis=0)

kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=seed)
cvscores = []

for train, test in kfold.split(X, Y):
    X_train = X[train]
    X_test = X[test]
    
    # preprocessing
    Y_train = np.eye(n_classes)[Y[train]]
    Y_test = np.eye(n_classes)[Y[test]]
    
    # bottleneck feature extraction
    train_datagen = image_generator(X_train, Y_train)
    bottleneck_features_training = pretrained_model.predict_generator(train_datagen(), len(X_train)//batch_size)

    test_datagen = image_generator(X_test, Y_test)
    bottleneck_features_testing = pretrained_model.predict_generator(test_datagen(), len(X_test)//batch_size)

    # Create model
    model = Sequential()
    model.add(Flatten(input_shape=bottleneck_features_training.shape[1:]))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    
    # Compile model
    model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Fit model
    history = model.fit(bottleneck_features_training, Y_train, 
          epochs=n_epochs,
          batch_size=batch_size,
          validation_data=(bottleneck_features_testing, Y_test),
          verbose=2)
    
    # Test accuracy
    scores = model.evaluate(bottleneck_features_testing, Y_test, batch_size=batch_size)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1]*100)
    
print("Cross validation scores:")
print(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
 - 14s - loss: 2.9091 - acc: 0.3098 - val_loss: 1.5304 - val_acc: 0.6132
Epoch 2/30
 - 14s - loss: 1.6069 - acc: 0.5527 - val_loss: 1.1838 - val_acc: 0.6710
Epoch 3/30
 - 14s - loss: 1.3228 - acc: 0.6213 - val_loss: 1.0640 - val_acc: 0.6950
Epoch 4/30
 - 13s - loss: 1.1841 - acc: 0.6560 - val_loss: 0.9984 - val_acc: 0.7076
Epoch 5/30
 - 14s - loss: 1.0907 - acc: 0.6819 - val_loss: 0.9534 - val_acc: 0.7173
Epoch 6/30
 - 14s - loss: 1.0176 - acc: 0.7004 - val_loss: 0.9258 - val_acc: 0.7241
Epoch 7/30
 - 14s - loss: 0.9650 - acc: 0.7187 - val_loss: 0.9026 - val_acc: 0.7317
Epoch 8/30
 - 14s - loss: 0.9182 - acc: 0.7292 - val_loss: 0.8850 - val_acc: 0.7357
Epoch 9/30
 - 14s - loss: 0.8855 - acc: 0.7366 - val_loss: 0.8691 - val_acc: 0.7367
Epoch 10/30
 - 14s - loss: 0.8480 - acc: 0.7474 - val_loss: 0.8608 - val_acc: 0.7432
Epoch 11/30
 - 14s - loss: 0.8226 - acc: 0.7548 - val_loss: 0.8462 - val_acc: 0.7478
Epoch 12/30
 - 14s - los

Epoch 3/30
 - 13s - loss: 1.3212 - acc: 0.6242 - val_loss: 1.0663 - val_acc: 0.6943
Epoch 4/30
 - 13s - loss: 1.1777 - acc: 0.6597 - val_loss: 1.0022 - val_acc: 0.7075
Epoch 5/30
 - 13s - loss: 1.0858 - acc: 0.6849 - val_loss: 0.9602 - val_acc: 0.7139
Epoch 6/30
 - 13s - loss: 1.0160 - acc: 0.7015 - val_loss: 0.9318 - val_acc: 0.7232
Epoch 7/30
 - 13s - loss: 0.9598 - acc: 0.7186 - val_loss: 0.9084 - val_acc: 0.7295
Epoch 8/30
 - 13s - loss: 0.9155 - acc: 0.7309 - val_loss: 0.8911 - val_acc: 0.7316
Epoch 9/30
 - 13s - loss: 0.8795 - acc: 0.7409 - val_loss: 0.8813 - val_acc: 0.7363
Epoch 10/30
 - 13s - loss: 0.8489 - acc: 0.7512 - val_loss: 0.8679 - val_acc: 0.7392
Epoch 11/30
 - 13s - loss: 0.8143 - acc: 0.7589 - val_loss: 0.8600 - val_acc: 0.7428
Epoch 12/30
 - 13s - loss: 0.7958 - acc: 0.7666 - val_loss: 0.8571 - val_acc: 0.7451
Epoch 13/30
 - 13s - loss: 0.7710 - acc: 0.7726 - val_loss: 0.8495 - val_acc: 0.7465
Epoch 14/30
 - 13s - loss: 0.7451 - acc: 0.7801 - val_loss: 0.8410 - val

References
- https://keras.io/applications/
- The Keras Blog https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
- http://nbviewer.jupyter.org/github/fchollet/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb
- Learn OpenCV https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/
- Cifar10 classification experiments https://github.com/rnoxy/cifar10-cnn